# Load Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
from project_lib import Project

#sklearn 
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
import os, types
from sklearn.metrics import f1_score
import numpy as np
import joblib
import datetime
import tempfile

#WML
from ibm_watson_machine_learning import APIClient

# Load Data

In [2]:

import itc_utils.flight_service as itcfs
readClient = itcfs.get_flight_client()

nb_data_request = {
    'data_name': """modeling_records_2022.csv"""
    }


flightInfo = itcfs.get_flight_info(readClient, nb_data_request=nb_data_request)

df = itcfs.read_pandas_and_concat(readClient, flightInfo, timeout=240)
df.head(10)

,POSITION_CODE,DEPARTMENT_CODE,DAYS_WITH_COMPANY,COMMUTE_TIME,AGE_BEGIN_PERIOD,GENDER_CODE,ATTRITION,PERIOD_TOTAL_DAYS,STARTING_SALARY,ENDING_SALARY,...,VACATION_DAYS_TAKEN,SICK_DAYS_TAKEN,PROMOTIONS,NB_MANAGERS,DAYS_IN_POSITION,DAYS_SINCE_LAST_RAISE,RANKING_CODE,OVERTIME,DBLOVERTIME,TRAVEL
0,1200,200,1825,29,55,0,0,330,159230.77,161538.46,...,28,10.5,0,1,1825,0,3,0,0,0
1,1200,200,2615,0,49,0,0,180,181153.85,183846.15,...,15,4,0,1,2615,60,3,0,0,0
2,1300,320,1609,30,44,1,0,330,129692.31,132923.08,...,20,15,0,1,1609,150,3,0,0,0
3,1300,320,2035,13,45,0,0,330,146769.23,150461.54,...,28,8,0,1,2035,210,3,0,0,0
4,1400,380,1885,31,44,0,0,330,146769.23,150461.54,...,26,11.5,0,1,1885,60,3,0,0,0
5,1400,380,3070,35,40,0,0,330,157846.15,161538.46,...,19,12.5,0,1,3070,150,3,0,0,0
6,1400,380,1825,9,40,1,0,330,146769.23,150461.54,...,20,9,0,1,1825,0,3,0,0,0
7,1500,300,2035,11,46,0,0,330,132923.08,136153.85,...,10,9,0,1,2035,210,3,0,0,0
8,1500,300,2250,0,45,1,0,180,150461.54,154153.85,...,10,4,0,1,2250,60,3,0,0,0
9,1600,330,1520,31,46,0,0,330,143076.92,146769.23,...,26,9.5,0,1,1520,60,3,0,0,0


In [3]:
from sklearn.model_selection import train_test_split

X = np.array(df.drop(['ATTRITION'], axis=1))  # Features
y = np.array(df['ATTRITION'])  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15) # 85% training and 15% test

In [4]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf = RandomForestClassifier(n_estimators=50)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

In [5]:
from sklearn import metrics
import pandas as pd
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9166666666666666


### Setup WML Client

In [6]:
# @hidden_cell
username = 'ritchie'
password = 'Enigma'
url = 'https://internal-nginx-svc:12443'

wml_credentials = {
    "username": username,
    "password": password,
    "url": url,
    "instance_id": 'openshift',
    "version": '4.6'
}

from ibm_watson_machine_learning import APIClient
wml_client = APIClient(wml_credentials)

In [9]:
# Get the Project ID and set the location to save the model to the project
import os
project_id = os.environ['PROJECT_ID']
wml_client.set.default_project(project_id)

'SUCCESS'

In [10]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.1-py3.9")
print("Software Specification ID: {}".format(software_spec_uid))

metadata = {
            wml_client.repository.ModelMetaNames.NAME: 'Random Forrest',
            wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
            wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

print("Storing model...")
published_model_details = wml_client.repository.store_model(
    model=clf, 
    meta_props=metadata,
    training_target=['ATTRITION'],
    training_data=X)
model_uid = wml_client.repository.get_model_id(published_model_details)

print("Done")
print("Model ID: {}".format(model_uid))

Software Specification ID: 12b83a17-24d8-5082-900f-0ab31fbfd3cb
Storing model...
Done
Model ID: d7c41ff9-bc70-4cbb-aad2-3cf827ce04ed


In [11]:
#Show all deployed ML Models
models_details = wml_client.repository.list_models()

------------------------------------  --------------------------------------------  ------------------------  ----------------
ID                                    NAME                                          CREATED                   TYPE
d7c41ff9-bc70-4cbb-aad2-3cf827ce04ed  Random Forrest                                2022-12-29T09:09:24.002Z  scikit-learn_1.0
45b3825f-7e33-4ffe-b5d6-8e137d7f0dd8  Random Forrest                                2022-12-28T18:54:26.002Z  scikit-learn_1.0
bbf516e8-144e-4363-8320-2c53e400271e  AutoAI - P5 Snap Boosting Machine Classifier  2022-12-27T16:41:55.002Z  wml-hybrid_0.1
c4f9f0c7-e4d1-4cf1-bed5-32c961e72d07  Decision Tree                                 2022-12-27T15:45:15.002Z  scikit-learn_1.0
------------------------------------  --------------------------------------------  ------------------------  ----------------


# Prepare Scoring Payload

In [9]:
fields = df.drop(['ATTRITION'], axis=1).columns.tolist()
fields

['POSITION_CODE',
 'DEPARTMENT_CODE',
 'DAYS_WITH_COMPANY',
 'COMMUTE_TIME',
 'AGE_BEGIN_PERIOD',
 'GENDER_CODE',
 'PERIOD_TOTAL_DAYS',
 'STARTING_SALARY',
 'ENDING_SALARY',
 'NB_INCREASES',
 'BONUS',
 'NB_BONUS',
 'VACATION_DAYS_TAKEN',
 'SICK_DAYS_TAKEN',
 'PROMOTIONS',
 'NB_MANAGERS',
 'DAYS_IN_POSITION',
 'DAYS_SINCE_LAST_RAISE',
 'RANKING_CODE',
 'OVERTIME',
 'DBLOVERTIME',
 'TRAVEL']

In [10]:
# Select one customer (as Numpy Array)
input = X_test[1]
input

array(['3900', '300', '910', '24', '44', '1', '330', '29615.38',
       '31730.77', '1', '1269.2308', '1', '15', '2', '0', '1', '910',
       '180', '3', '0', '0', '0'], dtype=object)

In [11]:
# Predict
clf.predict(input.reshape(-1, 22))

array(['0'], dtype=object)

In [12]:
# Predict Credit Default
clf.predict_proba(input.reshape(-1, 22))[:,1]

array([0.16])

In [13]:
import json
scoring_values = input.tolist()
scoring_payload = {"input_data": [{"values": [scoring_values]}]}
print(json.dumps(scoring_payload))

{"input_data": [{"values": [["3900", "300", "910", "24", "44", "1", "330", "29615.38", "31730.77", "1", "1269.2308", "1", "15", "2", "0", "1", "910", "180", "3", "0", "0", "0"]]}]}


# Realtime Scoring

In [14]:
!pip list | grep watson

ibm-watson-machine-learning   1.0.253
ibm-watson-openscale          3.0.21
ibm-watson-studio-lib         3.0.10


In [12]:
import requests
url     = wml_credentials["url"]+'/v1/preauth/validateAuth'
headers = { "Content-Type" : "application/x-www-form-urlencoded" }
username = wml_credentials["username"]
password = wml_credentials["password"]
response  = requests.get( url,  auth=( username, password ) , verify=False)
response.json()
mltoken = response.json()["accessToken"]
mltoken

'eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6InJxRjItMjdRMS1NQWNlZ3Y1WTk2R2dkWlFtVzZYaVRqNmJJSWxnQUtIV3MifQ.eyJ1c2VybmFtZSI6InJpdGNoaWUiLCJyb2xlIjoiQWRtaW4iLCJwZXJtaXNzaW9ucyI6WyJjcmVhdGVfcHJvamVjdCIsImFjY2Vzc19jYXRhbG9nIiwidmlld19xdWFsaXR5IiwiY3JlYXRlX3NwYWNlIiwibWFuYWdlX21ldGFkYXRhX2ltcG9ydCIsIm1hbmFnZV9kaXNjb3ZlcnkiLCJtYW5hZ2VfcXVhbGl0eSIsInZpZXdfZ292ZXJuYW5jZV9hcnRpZmFjdHMiLCJhdXRob3JfZ292ZXJuYW5jZV9hcnRpZmFjdHMiLCJtYW5hZ2VfcmVwb3J0aW5nIiwibW9uaXRvcl9wcm9qZWN0IiwiY2FuX3Byb3Zpc2lvbiIsInNpZ25faW5fb25seSIsImFkbWluaXN0cmF0b3IiLCJtYW5hZ2VfdmF1bHRzX2FuZF9zZWNyZXRzIiwic2hhcmVfc2VjcmV0cyIsImFkZF92YXVsdHMiLCJtYW5hZ2VfY2F0YWxvZyIsIm1hbmFnZV9nb3Zlcm5hbmNlX3dvcmtmbG93IiwibWFuYWdlX2NhdGVnb3JpZXMiLCJtYW5hZ2VfZ2xvc3NhcnkiLCJhY2Nlc3NfZGF0YV9xdWFsaXR5X2Fzc2V0X3R5cGVzIl0sImdyb3VwcyI6WzEwMDAxLDEwMDAyLDEwMDAwXSwic3ViIjoicml0Y2hpZSIsImlzcyI6IktOT1hTU08iLCJhdWQiOiJEU1giLCJ1aWQiOiIxMDAwMzMxMDA3IiwiYXV0aGVudGljYXRvciI6ImRlZmF1bHQiLCJkaXNwbGF5X25hbWUiOiJBbGV4YW5kZXIgUmljaHRoYW1tZXIiLCJpYXQiOjE2NzIzMDUyODIs

In [ ]:
import urllib3, requests, json

# NOTE: you must construct mltoken based on provided documentation
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"input_data": [{"fields": fields, "values": scoring_values}]}

response_scoring = requests.post('https://ai-trust.cp4d-demos.com/ml/v4/deployments/59d07b60-78cf-447d-ab60-184c904f796f/predictions?version=2022-12-28', json=payload_scoring, headers=header)
print("Scoring response")
print(json.loads(response_scoring.text))
